Weak Pin Bar: The volumes on a pin bar are important but the most important thing which determines if the pin bar is weak or not is the follow up bar and it's volume.

Hypothesis: If the follow up bar gives greater move than the pin bar but the volume was lower than the pin bar then, the low of the pin bar should get tested. This testing depends on the time frame of the pin bar. The results can be seen more quickly on intraday time frames so, we will be using 3, 5, and 15 minutes time frames on BTCUSDT and ETHUSDT futures contracts.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
btcusdt_15min_data = pd.read_csv("BTCUSDT_15min.csv")
btcusdt_15min_df = pd.DataFrame(btcusdt_15min_data)
btcusdt_15min_df.head()

,timestamp,open,high,low,close,volume
0,2022-07-04 20:15:00,19851.3,19853.6,19719.5,19776.6,4522.198
1,2022-07-04 20:30:00,19776.7,19779.3,19735.5,19758.5,2417.909
2,2022-07-04 20:45:00,19758.5,19809.2,19745.7,19774.4,2972.344
3,2022-07-04 21:00:00,19774.3,19790.0,19740.4,19789.9,3271.586
4,2022-07-04 21:15:00,19789.9,19811.4,19768.3,19773.8,2151.420
5,2022-07-04 21:30:00,19773.8,19979.7,19768.0,19870.9,6511.104
6,2022-07-04 21:45:00,19870.9,20033.0,19870.3,19946.7,5781.626
7,2022-07-04 22:00:00,19946.7,19960.3,19865.5,19873.6,5024.952
8,2022-07-04 22:15:00,19873.7,19970.0,19863.3,19943.1,3689.065
9,2022-07-04 22:30:00,19943.2,20100.0,19926.4,20016.5,10324.172


Checking for bullish pin bars on 15 minutes bars of BTCUSDT Futures

In [23]:
# Defining function to identify bullish pin bars
def is_bullish_pin_bar(row):
    o_price = row['open']
    h_price = row['high']
    l_price = row['low']
    c_price = row['close']

    body = abs(c_price - o_price)
    upper_wick = h_price - max(c_price, o_price)
    lower_wick = min(o_price, c_price) - l_price
    price_range = h_price - l_price
    body_ratio = body/price_range

    if(c_price >= o_price and price_range > 0):
        if(upper_wick < lower_wick and body_ratio < 0.5):
            return True
        else:
            return False
    return False

btcusdt_15min_df['bullish_pin_bar'] = btcusdt_15min_df.apply(is_bullish_pin_bar, axis=1)
btcusdt_15min_df.head()

,timestamp,open,high,low,close,volume,bullish_pin_bar
0,2022-07-04 20:15:00,19851.3,19853.6,19719.5,19776.6,4522.198,False
1,2022-07-04 20:30:00,19776.7,19779.3,19735.5,19758.5,2417.909,False
2,2022-07-04 20:45:00,19758.5,19809.2,19745.7,19774.4,2972.344,False
3,2022-07-04 21:00:00,19774.3,19790.0,19740.4,19789.9,3271.586,True
4,2022-07-04 21:15:00,19789.9,19811.4,19768.3,19773.8,2151.420,False
